In [ ]:
import os
import pandas as pd

path = 'c:/Users/soirk/Krisztian/Egyetem/Survey Statisztika Msc/Szakdolgozat/corpus/'

papers_a= pd.read_csv(path+'arxiv.csv')
papers_j= pd.read_csv(path+'jstor_metadata.csv')

papers_j=papers_j.drop(columns=['journal_title'])
papers_j.columns = ['id','title','date','category']

papers=pd.concat([papers_a,papers_j])

In [ ]:
from nltk import word_tokenize

titles_token = []
for title in papers['title']:
    titles_token.append(word_tokenize(str(title)))

In [ ]:
from nltk.stem import PorterStemmer
ps=PorterStemmer()

titles_stemmed=[]

for i in range(len(titles_token)):
    words_stemmed=[]
    for words in titles_token[i]:
        words_stemmed.append(ps.stem(words))
    titles_stemmed.append(words_stemmed)

In [ ]:
from nltk.probability import FreqDist

title_fdist=[]

miss=0
data=0
handl=0
obs=0
method=0
incomp=0

for title in titles_stemmed:
    title_fdist.append(dict(FreqDist(title)))
    if 'miss' in dict(FreqDist(title)):
        miss+=1
    elif 'handl' in dict(FreqDist(title)):
        handl+=1
    elif 'data' in dict(FreqDist(title)):
        data+=1
    elif 'observ' in dict(FreqDist(title)):
        obs+=1
    elif 'method' in dict(FreqDist(title)):
        method+=1
    elif 'incomplet' in dict(FreqDist(title)):
        incomp+=1
    else:
        continue

papers['title']=titles_stemmed

In [ ]:
def weighter(word_names,word_list):
    weight_list=[]
    for name,word in zip(word_names,word_list):
        weight_list.append([name,round(1/word*100,3)])
    return(weight_list)


words=[miss,data,handl,obs,method,incomp]
names=['miss','data','handl','obs','method','incomp']

weights=weighter(names,words)

In [ ]:
from nltk.probability import FreqDist

miss_list=[]
data_list=[]
handl_list=[]
obs_list=[]
method_list=[]
incomp_list=[]


for title in papers['title']:
    f=FreqDist(title)
    if 'miss' in f:
        miss_list.append(f['miss']*weights[0][1])
    else:
        miss_list.append(0)
    if 'data' in f:
        data_list.append(f['data']*weights[1][1])
    else:
        data_list.append(0)
    if 'handl' in f:
        handl_list.append(f['handl']*weights[2][1])
    else:
        handl_list.append(0)
    if 'observ' in f:
        obs_list.append(f['observ']*weights[3][1])
    else:
        obs_list.append(0)
    if 'method' in f:
        method_list.append(f['method']*weights[4][1])
    else:
        method_list.append(0)
    if 'incomplet' in f:
        incomp_list.append(f['incomplet']*weights[5][1])
    else:
        incomp_list.append(0)

In [ ]:
word_weights=pd.DataFrame(list(zip(miss_list,data_list,handl_list,obs_list,method_list,incomp_list)),columns=['miss','data','handl','obs','method','incomp'])

In [ ]:
word_weights=word_weights.reset_index(drop=True)
papers=papers.reset_index(drop=True)

In [ ]:
papers_w=pd.concat([papers,word_weights],axis=1)

In [ ]:
papers_w.to_csv(r'c:\Users\soirk\Krisztian\Egyetem\Survey Statisztika Msc\Szakdolgozat\corpus\papers_w.csv',index=False)